<a href="https://colab.research.google.com/github/sergfer26/Proyectos-II/blob/main/semana2/clase5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print('esto es una prueba')

esto es una prueba
